## Loading Data from Kaggle server to Colab (Google server)
feel free to remove this cell if you are not using colab 

**but you MUST download the dataset(put in the folder data/) and the glove embedding**

In [ ]:
from google.colab import files
files.upload()
#choose kaggle.json
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d abhinavwalia95/entity-annotated-corpus
!mkdir content/data/
!unzip entity-annotated-corpus.zip -d data/


#download Glove
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

Saving kaggle.json to kaggle.json
 64% 17.0M/26.4M [00:00<00:00, 72.2MB/s]
100% 26.4M/26.4M [00:00<00:00, 88.3MB/s]
mkdir: cannot create directory ‘content/data/’: No such file or directory
Archive:  entity-annotated-corpus.zip
  inflating: data/ner.csv            
  inflating: data/ner_dataset.csv    
--2021-03-23 18:26:54--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-03-23 18:26:54--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-03-23 18:26:55--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip

# Starting of the solution 

### importing important utilities 

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from random import shuffle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
glove = {}
f = open('glove.6B.50d.txt', encoding='utf-8')

print("Loading Glove Model")
for line in f:
  splitLines = line.split()
  word = splitLines[0]
  wordEmbedding = np.array([float(value) for value in splitLines[1:]])
  glove[word] = wordEmbedding



Loading Glove Model


### Data preprocessing

In [ ]:
DATA_PATH =r"data/ner_dataset.csv"
data = pd.read_csv(DATA_PATH , encoding = 'latin1')
print(data.head())
data = data.drop(['POS'] , axis =1 )
# data = data.dropna()
data = data.fillna(method="ffill")
print(data.head())

    Sentence #           Word  POS Tag
0  Sentence: 1      Thousands  NNS   O
1          NaN             of   IN   O
2          NaN  demonstrators  NNS   O
3          NaN           have  VBP   O
4          NaN        marched  VBN   O
    Sentence #           Word Tag
0  Sentence: 1      Thousands   O
1  Sentence: 1             of   O
2  Sentence: 1  demonstrators   O
3  Sentence: 1           have   O
4  Sentence: 1        marched   O


implementing the function that gets and groups data into sentences 

In [ ]:
def getSentences(data):
    """
    This class prepares the Data then gets the sentence after sentence 
    """
    agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),s["Tag"].values.tolist())]
    data_grouped = data.groupby("Sentence #").apply(agg_func)
    data_sentences = [s for s in data_grouped]
    return data_sentences

    

In [ ]:
#words is set of all usique words (vocab)
words = set(list(data['Word'].values))
words.add('PADDING')
n_words = len(words)

# set of all unique tags 
tags = list(set(data["Tag"].values))
n_tags = len(tags)


# mapping from word to id 
words2index = {w:i for i,w in enumerate(words)}
#mapping from tag to id
tags2index = {t:i for i,t in enumerate(tags)}
index2tag = {i:t for i,t in enumerate(tags)}


sentences = getSentences(data)
sentences[0]


[('Thousands', 'O'),
 ('of', 'O'),
 ('demonstrators', 'O'),
 ('have', 'O'),
 ('marched', 'O'),
 ('through', 'O'),
 ('London', 'B-geo'),
 ('to', 'O'),
 ('protest', 'O'),
 ('the', 'O'),
 ('war', 'O'),
 ('in', 'O'),
 ('Iraq', 'B-geo'),
 ('and', 'O'),
 ('demand', 'O'),
 ('the', 'O'),
 ('withdrawal', 'O'),
 ('of', 'O'),
 ('British', 'B-gpe'),
 ('troops', 'O'),
 ('from', 'O'),
 ('that', 'O'),
 ('country', 'O'),
 ('.', 'O')]

In [ ]:
max_len = 50
new_X = []
for sentence in sentences:
        new_seq = []
        for i in range(max_len):
            try:
                new_seq.append(sentence[i])
            except:
                new_seq.append(("PADDING" , 'O'))
        new_X.append(new_seq)
new_X[13]

[('Syria', 'B-geo'),
 ('is', 'O'),
 ('widely', 'O'),
 ('accused', 'O'),
 ('of', 'O'),
 ('involvement', 'O'),
 ('in', 'O'),
 ('his', 'O'),
 ('killing', 'O'),
 (',', 'O'),
 ('and', 'O'),
 ('Friday', 'B-tim'),
 ("'s", 'O'),
 ('explosion', 'O'),
 ('comes', 'O'),
 ('days', 'O'),
 ('before', 'O'),
 ('U.N.', 'B-geo'),
 ('investigator', 'O'),
 ('Detlev', 'B-per'),
 ('Mehlis', 'I-per'),
 ('is', 'O'),
 ('to', 'O'),
 ('return', 'O'),
 ('to', 'O'),
 ('Damascus', 'B-geo'),
 ('to', 'O'),
 ('interview', 'O'),
 ('several', 'O'),
 ('Syrian', 'B-gpe'),
 ('officials', 'O'),
 ('about', 'O'),
 ('the', 'O'),
 ('assassination', 'O'),
 ('.', 'O'),
 ('PADDING', 'O'),
 ('PADDING', 'O'),
 ('PADDING', 'O'),
 ('PADDING', 'O'),
 ('PADDING', 'O'),
 ('PADDING', 'O'),
 ('PADDING', 'O'),
 ('PADDING', 'O'),
 ('PADDING', 'O'),
 ('PADDING', 'O'),
 ('PADDING', 'O'),
 ('PADDING', 'O'),
 ('PADDING', 'O'),
 ('PADDING', 'O'),
 ('PADDING', 'O')]

In [ ]:
shuffle(new_X)
tr = int(0.8*len(new_X))
train_data =new_X[:tr]
test_data =new_X[tr:]


In [ ]:
new_X =[]
for sent in train_data :
  xseq =[]
  yseq =[]
  for word , tag in sent :
    onehot_tag = torch.zeros(n_tags)
    onehot_tag[tags2index[tag]]=1
    try :
      xseq.append(glove[word.lower()])
      yseq.append(onehot_tag.tolist())
    except :
        xseq.append(np.zeros(50))
        yseq.append(onehot_tag.tolist())
  new_X.append((xseq , yseq))
new_X[0]

([array([ 0.25374 , -0.13279 ,  0.39656 ,  1.1883  , -0.36759 , -0.70395 ,
          0.074684,  0.46931 , -0.092717,  1.2899  ,  0.84232 , -0.28645 ,
          0.42057 ,  0.17309 ,  0.42469 , -0.45683 , -0.24398 , -0.085802,
          0.36572 ,  0.020408, -0.50871 , -0.65908 ,  0.22688 , -0.2535  ,
         -0.73554 , -1.4334  ,  0.8889  ,  0.14779 ,  0.34656 ,  0.28487 ,
          1.9534  ,  0.15365 , -0.8714  , -0.12061 , -0.76205 , -0.77002 ,
         -1.3829  ,  0.10867 ,  0.039984,  0.012471, -0.30518 ,  0.75282 ,
          1.191   , -1.4656  , -0.76304 ,  0.96938 , -0.97976 ,  0.17433 ,
         -0.32077 , -0.66441 ]),
  array([ 6.1850e-01,  6.4254e-01, -4.6552e-01,  3.7570e-01,  7.4838e-01,
          5.3739e-01,  2.2239e-03, -6.0577e-01,  2.6408e-01,  1.1703e-01,
          4.3722e-01,  2.0092e-01, -5.7859e-02, -3.4589e-01,  2.1664e-01,
          5.8573e-01,  5.3919e-01,  6.9490e-01, -1.5618e-01,  5.5830e-02,
         -6.0515e-01, -2.8997e-01, -2.5594e-02,  5.5593e-01,  2.5356e-0

## non-changing variables and functions 

In [ ]:
inputsz = 50
hiddensz= 64
layers = 2
epochs = 2
learning_rate = 0.0005
batch_size =10

## LSTM 

### LSTM class

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_entities):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # -> x needs to be: (batch_size, seq, input_size)

        #additional layer to convert hidden state to output
        self.fully_connected = nn.Linear(hidden_size, num_entities)
        self.smax = nn.Softmax(dim =2)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        out, _ = self.lstm(x, (h0,c0))  
        # print(f'out shape {out.size()}')
        output =[]
       # for i in range(out.size()[1]):
        res =self.fully_connected(out)
        # res= self.smax(res)
        # res = torch.argmax(res , dim =2)
        # print(f'result shape is {res.shape}')
        return res


### Now we start to train our model 

In [ ]:
step =0
model = LSTM(inputsz, hiddensz, layers, n_tags).to(device)
cost = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  
sample =torch.tensor([new_X[0][0]  , new_X[1][0] ]  , dtype=torch.float).to(device)
for epoch in range(epochs):
    for i in range(0, len(new_X) , batch_size):
      try :
        input_batch =[new_X[z][0] for z in range( i , i+ batch_size)]
        labels =[new_X[z][1] for z in range( i , i+ batch_size)]
      except :
        break 
      input_batch = torch.tensor(input_batch , dtype= torch.float).to(device)
      labels = torch.tensor(labels  , dtype= torch.float).to(device)
      outputs = model(input_batch)
      Loss= cost(outputs , labels)
      # print(f"labels shape {labels.shape}")
      # print(outputs)

      optimizer.zero_grad()
      Loss.backward()
      optimizer.step()
      step +=1
      if step %100==0:
        print(f'step :{step } , loss = {Loss.item():.4f}')



step :100 , loss = 0.0088
step :200 , loss = 0.0068
step :300 , loss = 0.0055
step :400 , loss = 0.0058
step :500 , loss = 0.0049
step :600 , loss = 0.0037
step :700 , loss = 0.0058
step :800 , loss = 0.0051
step :900 , loss = 0.0049
step :1000 , loss = 0.0071
step :1100 , loss = 0.0039
step :1200 , loss = 0.0026
step :1300 , loss = 0.0039
step :1400 , loss = 0.0050
step :1500 , loss = 0.0042
step :1600 , loss = 0.0024
step :1700 , loss = 0.0034
step :1800 , loss = 0.0030
step :1900 , loss = 0.0026
step :2000 , loss = 0.0021
step :2100 , loss = 0.0031
step :2200 , loss = 0.0022
step :2300 , loss = 0.0047
step :2400 , loss = 0.0023
step :2500 , loss = 0.0021
step :2600 , loss = 0.0027
step :2700 , loss = 0.0017
step :2800 , loss = 0.0031
step :2900 , loss = 0.0027
step :3000 , loss = 0.0027
step :3100 , loss = 0.0034
step :3200 , loss = 0.0029
step :3300 , loss = 0.0026
step :3400 , loss = 0.0011
step :3500 , loss = 0.0031
step :3600 , loss = 0.0034
step :3700 , loss = 0.0034
step :3800

### testing model 

In [ ]:
correct , all  , step= 0,0,0
for test in test_data :
      test_seq = [] 
      words= []
      tags =[]
      out_tags =[]
      for word , tag in test :
        words.append(word)
        tags.append(tag)
        try :
          test_seq.append(glove[word.lower()])
        except :
          test_seq.append(np.zeros(50))
      
      input_test = torch.tensor([test_seq] , dtype= torch.float).to(device)
      out = torch.argmax(model(input_test) , dim =2)
      out = out.tolist()[0]
      # print(out)
      out_tags = [index2tag[x] for x in out ]
      # print(tags )
      # print(out_tags)
      for i in range (len(tags)):
          if tags[i] == out_tags[i]:
            correct +=1
          all +=1
      step+=1
      if step % 100 ==0 :
        print(f"==== Test number {step } ====")
        print(words)
        print(f"correct tags {tags}")
        print(f"output  tags {out_tags}")
  
print(f"over all accuracy = { (correct/ all )*100}%") 

==== Test number 100 ====
['Minutes', 'later', ',', '248', 'lawmakers', 'voted', 'to', 'oust', 'Interior', 'Minister', 'Yuri', 'Lutsenko', '-', 'who', 'had', 'barely', 'survived', 'a', 'parliamentary', 'attempt', 'to', 'dismiss', 'him', 'Thursday', '.', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING']
correct tags ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-per', 'I-per', 'I-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
output  tags ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-per', 'I-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

##GRU 

### GRU Class 

In [ ]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_entities):
        super(GRU, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)

        #additional layer to convert hidden state to output
        self.fully_connected = nn.Linear(hidden_size, num_entities)
        self.smax = nn.Softmax(dim =2)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        # c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        out, _ = self.gru(x, h0)  
        # print(f'out shape {out.size()}')
        output =[]
        #for i in range(out.size()[1]):
        res =self.fully_connected(out)
        # res= self.smax(res)
        # res = torch.argmax(res , dim =2)
        # print(f'result shape is {res.shape}')
        return res



### Training Model

In [ ]:

step =0
model = GRU(inputsz, hiddensz, layers, n_tags).to(device)
cost = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  
sample =torch.tensor([new_X[0][0]  , new_X[1][0] ]  , dtype=torch.float).to(device)
for epoch in range(epochs):
    for i in range(0, len(new_X) , batch_size):
      try :
        input_batch =[new_X[z][0] for z in range( i , i+ batch_size)]
        labels =[new_X[z][1] for z in range( i , i+ batch_size)]
      except :
        break 
      input_batch = torch.tensor(input_batch , dtype= torch.float).to(device)
      labels = torch.tensor(labels  , dtype= torch.float).to(device)
      outputs = model(input_batch)
      Loss= cost(outputs , labels)
      optimizer.zero_grad()
      Loss.backward()
      optimizer.step()
      step +=1
      if step %5==0:
        print(f'step :{step } , loss = {Loss.item():.4f}')


step :5 , loss = 0.0414
step :10 , loss = 0.0190
step :15 , loss = 0.0111
step :20 , loss = 0.0071
step :25 , loss = 0.0082
step :30 , loss = 0.0062
step :35 , loss = 0.0110
step :40 , loss = 0.0082
step :45 , loss = 0.0080
step :50 , loss = 0.0111
step :55 , loss = 0.0083
step :60 , loss = 0.0057
step :65 , loss = 0.0094
step :70 , loss = 0.0069
step :75 , loss = 0.0053
step :80 , loss = 0.0068
step :85 , loss = 0.0060
step :90 , loss = 0.0071
step :95 , loss = 0.0058
step :100 , loss = 0.0081
step :105 , loss = 0.0067
step :110 , loss = 0.0075
step :115 , loss = 0.0049
step :120 , loss = 0.0072
step :125 , loss = 0.0044
step :130 , loss = 0.0085
step :135 , loss = 0.0068
step :140 , loss = 0.0083
step :145 , loss = 0.0082
step :150 , loss = 0.0066
step :155 , loss = 0.0077
step :160 , loss = 0.0086
step :165 , loss = 0.0060
step :170 , loss = 0.0062
step :175 , loss = 0.0044
step :180 , loss = 0.0060
step :185 , loss = 0.0084
step :190 , loss = 0.0056
step :195 , loss = 0.0057
step :

### Testing Model

In [ ]:
correct , all  , step= 0,0,0
for test in test_data :
      test_seq = [] 
      words= []
      tags =[]
      out_tags =[]
      for word , tag in test :
        words.append(word)
        tags.append(tag)
        try :
          test_seq.append(glove[word.lower()])
        except :
          test_seq.append(np.zeros(50))
      
      input_test = torch.tensor([test_seq] , dtype= torch.float).to(device)
      out = torch.argmax(model(input_test) , dim =2)
      out = out.tolist()[0]
      # print(out)
      out_tags = [index2tag[x] for x in out ]
      # print(tags )
      # print(out_tags)
      for i in range (len(tags)):
          if tags[i] == out_tags[i]:
            correct +=1
          all +=1
      step+=1
      if step % 1000 ==0 :
        print(f"==== Test number {step } ====")
        print(words)
        print(f"correct tags {tags}")
        print(f"output  tags {out_tags}")
  
print(f"over all accuracy = { (correct/ all )*100}%") 

==== Test number 1000 ====
['NATO', 'officials', 'in', 'Afghanistan', 'say', 'one', 'of', 'their', 'bombs', 'missed', 'its', 'target', 'in', 'eastern', 'Khost', 'province', 'Tuesday', 'and', 'instead', 'killed', 'two', 'civilians', 'and', 'wounded', 'up', 'to', '10', 'others', '.', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING', 'PADDING']
correct tags ['B-org', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
output  tags ['B-org', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'I-geo', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '